# NY Motor Vehicle Collisions – Exploratory Data Analysis  
In this notebook, we analyze New York’s Vehicular crash data available at https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95


The data contains information from all police reported motor vehicle collisions in NYC. The information for this dataset is collated from the police report, called MV104-AN, which is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage.

Data is available from 2012-07-01 onwards, however for this analysis, we will limit ourselves to the period up to 2023-08-15, which is when the data was downloaded.

We find that this data contains over 2 million observations, which allows us to explore several aspects of vehicle crashes in NY's boroughs.

The data dictionary for the data is also available at the URL above.

First, we perform unstructured exploration of the data, and then try to answer the following questions:

- We look for which borough has had the maximum number of crashes reported since 2012.

- We relate the number of crashes to the borough's population to find out which borough has the maximum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website (https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

| Borough | Population |
| --- | --- |
| Bronx | 1446788 |
| Brooklyn | 2648452 |
| Manhattan | 1638281 |
| Queens | 2330295 |
| Staten Island | 487155 |  

  
- We look for the leading cause of crashes

- We also look for the top-3 causes of crashes, and try to calculate what proportion of all crashes are caused by these top-3 causes.

- We then look to some of the more serious implications of crashes by examining how many accidents involved at least one fatality.

- We then compute, on average, out of every 1000 accidents, how many have resulted in at least one person dead.

- We also look for missing data and try to compute the proportion of accidents in the data that do not have a Borough code.

- The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident.  We look for which combinations of vehicles have the most number of accidents.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import phik
from IPython.display import Markdown as md

In [3]:
df = pd.read_pickle("shared/Motor_Vehicle_Collisions_-_Crashes.pkl")
df


,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


***
### Show some exploratory data analysis here
First, we can see the information of the dataset for columns.
***

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018245 entries, 0 to 2018244
Data columns (total 28 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   CRASH DATE_CRASH TIME          datetime64[ns]
 1   BOROUGH                        object        
 2   ZIP CODE                       object        
 3   LATITUDE                       float64       
 4   LONGITUDE                      float64       
 5   LOCATION                       object        
 6   ON STREET NAME                 object        
 7   CROSS STREET NAME              object        
 8   OFF STREET NAME                object        
 9   NUMBER OF PERSONS INJURED      float64       
 10  NUMBER OF PERSONS KILLED       float64       
 11  NUMBER OF PEDESTRIANS INJURED  int64         
 12  NUMBER OF PEDESTRIANS KILLED   int64         
 13  NUMBER OF CYCLIST INJURED      int64         
 14  NUMBER OF CYCLIST KILLED       int64         
 15  NUMBER OF MOTOR

In [5]:
# Let us look at some descriptive statistics for the numerical variables
df.describe()

,CRASH DATE_CRASH TIME,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,COLLISION_ID
count,2018245,1.788560e+06,1.788560e+06,2.018227e+06,2.018214e+06,2.018245e+06,2.018245e+06,2.018245e+06,2.018245e+06,2.018245e+06,2.018245e+06,2.018245e+06
mean,2017-05-14 17:39:03.899198976,4.062776e+01,-7.375228e+01,3.024249e-01,1.446328e-03,5.518507e-02,7.253827e-04,2.612468e-02,1.119785e-04,2.179889e-01,5.896212e-04,3.116455e+06
min,2012-07-01 00:05:00,0.000000e+00,-2.013600e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.200000e+01
25%,2014-12-19 18:00:00,4.066792e+01,-7.397493e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.140681e+06
50%,2017-03-29 21:15:00,4.072102e+01,-7.392732e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.645346e+06
75%,2019-06-10 18:30:00,4.076956e+01,-7.386665e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.150156e+06
max,2023-08-15 23:59:00,4.334444e+01,0.000000e+00,4.300000e+01,8.000000e+00,2.700000e+01,6.000000e+00,4.000000e+00,2.000000e+00,4.300000e+01,5.000000e+00,4.655026e+06
std,NaN,1.980901e+00,3.727568e+00,6.937633e-01,4.007201e-02,2.412867e-01,2.741556e-02,1.614267e-01,1.062812e-02,6.549700e-01,2.648117e-02,1.503997e+06


In [17]:
# This identifies columns with missing values in each column.
missing_data = df.isnull().sum().sort_values(ascending = False)
missing_data

VEHICLE TYPE CODE 5              2009835
CONTRIBUTING FACTOR VEHICLE 5    2009575
VEHICLE TYPE CODE 4              1987193
CONTRIBUTING FACTOR VEHICLE 4    1986122
VEHICLE TYPE CODE 3              1880098
CONTRIBUTING FACTOR VEHICLE 3    1875114
OFF STREET NAME                  1685810
CROSS STREET NAME                 755532
ZIP CODE                          628092
BOROUGH                           627854
ON STREET NAME                    424807
VEHICLE TYPE CODE 2               376990
CONTRIBUTING FACTOR VEHICLE 2     307909
LONGITUDE                         229685
LOCATION                          229685
LATITUDE                          229685
VEHICLE TYPE CODE 1                12677
CONTRIBUTING FACTOR VEHICLE 1       6348
NUMBER OF PERSONS KILLED              31
NUMBER OF PERSONS INJURED             18
COLLISION_ID                           0
CRASH DATE_CRASH TIME                  0
NUMBER OF MOTORIST KILLED              0
NUMBER OF MOTORIST INJURED             0
NUMBER OF CYCLIS

## Borough with the maximum number of crashes reported since 2012

In [7]:
df["BOROUGH"].value_counts()

BOROUGH
BROOKLYN         441026
QUEENS           372457
MANHATTAN        313266
BRONX            205345
STATEN ISLAND     58297
Name: count, dtype: int64

In [8]:
borough_crashes = df["BOROUGH"].value_counts()
print('Borough with the maximum number of crashes',borough_crashes.idxmax())
print('The max number',borough_crashes.max())

Borough with the maximum number of crashes BROOKLYN
The max number 441026


## Borough with the maximum number of crashes adjusted for population

We relate the number of crashes to to the borough's population to find out which borough has the maximum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website (https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

Use the below population estimates for the question that follows:

| Borough | Population |
| --- | --- |
| Bronx | 1446788 |
| Brooklyn | 2648452 |
| Manhattan | 1638281 |
| Queens | 2330295 |
| Staten Island | 487155 |  

Considering the entire data set since July 2012, let's calculate which borough has the most crashes for every 100,000 people.

In [9]:
pop_df = pd.DataFrame({'BOROUGH':['BRONX','BROOKLYN','MANHATTAN','QUEENS','STATEN ISLAND'],
                      'POPULATION':[1446788,2648452,1638281,2330295,487155]}).set_index('BOROUGH')

pop_df

,POPULATION
BOROUGH,
BRONX,1446788
BROOKLYN,2648452
MANHATTAN,1638281
QUEENS,2330295
STATEN ISLAND,487155


In [10]:
combined_df = borough_crashes.reset_index()
combined_df.columns = ['BOROUGH','CRASH_COUNT']
combined_df = combined_df.merge(pop_df, on='BOROUGH')
combined_df

,BOROUGH,CRASH_COUNT,POPULATION
0,BROOKLYN,441026,2648452
1,QUEENS,372457,2330295
2,MANHATTAN,313266,1638281
3,BRONX,205345,1446788
4,STATEN ISLAND,58297,487155


In [11]:
# Calculate the crash rate per 100,000 people for each borough
combined_df['CRASH_RATE_PER_100K'] = (combined_df['CRASH_COUNT'] / combined_df['POPULATION']) * 100000
combined_df

,BOROUGH,CRASH_COUNT,POPULATION,CRASH_RATE_PER_100K
0,BROOKLYN,441026,2648452,16652.217975
1,QUEENS,372457,2330295,15983.255339
2,MANHATTAN,313266,1638281,19121.628097
3,BRONX,205345,1446788,14193.164444
4,STATEN ISLAND,58297,487155,11966.827806


## Analyzing the leading cause of crashes

For a large proportion of the crashes, the 'contributing factor vehicle 1' is a missing value, indicated as 'Unspecified' in the data.

Remove all rows where the contributing factor vehicle 1 is not known, leaving only the crashes where only this information is available.

In [12]:
df_cause = df[df['CONTRIBUTING FACTOR VEHICLE 1']!='Unspecified'].dropna(subset=['CONTRIBUTING FACTOR VEHICLE 1'])
df_cause['CONTRIBUTING FACTOR VEHICLE 1']

0            Aggressive Driving/Road Rage
1                       Pavement Slippery
2                   Following Too Closely
6                     Passing Too Closely
8                     Driver Inexperience
                        ...              
2018238             Following Too Closely
2018239                  Backing Unsafely
2018240                Turning Improperly
2018242    Driver Inattention/Distraction
2018243    Driver Inattention/Distraction
Name: CONTRIBUTING FACTOR VEHICLE 1, Length: 1319161, dtype: object

## Top 3 causes of crashes 

In [13]:
# Determine the leading cause of crashes
total_crash = len(df_cause)
leading_cause = df_cause['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().reset_index()
leading_cause.columns = ['CONTRIBUTING FACTOR VEHICLE 1','CAUSE_COUNT']
leading_cause['PROPORTION'] = leading_cause['CAUSE_COUNT']/total_crash
leading_cause

,CONTRIBUTING FACTOR VEHICLE 1,CAUSE_COUNT,PROPORTION
0,Driver Inattention/Distraction,401262,0.304180
1,Failure to Yield Right-of-Way,119166,0.090335
2,Following Too Closely,107467,0.081466
3,Backing Unsafely,75042,0.056886
4,Other Vehicular,62688,0.047521
5,Passing or Lane Usage Improper,55445,0.042031
6,Turning Improperly,49908,0.037833
7,Passing Too Closely,49848,0.037788
8,Fatigued/Drowsy,47343,0.035889
9,Unsafe Lane Changing,39711,0.030103


## Analyzing fatalities

From the previous EDA, we know that in the column 'NUMBER OF PERSONS KILLED', 'NUMBER OF MOTORIST KILLED ', 'NUMBER OF PEDESTRIANS KILLED ' and 'NUMBER OF CYCLIST KILLED'  represent the fatalities

In [27]:
df_fatalities = df
df_fatalities['FATALITIES'] = df['NUMBER OF PERSONS KILLED']+df['NUMBER OF MOTORIST KILLED']+df['NUMBER OF PEDESTRIANS KILLED']+df['NUMBER OF CYCLIST KILLED']
df

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,FATALITIES
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN,0.0
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN,0.0
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN,0.0
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,0.0
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN,0.0
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN,0.0
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN,0.0
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN,0.0


In [28]:

# Drop missing value
df_fatalities = df_fatalities[df_fatalities['FATALITIES']>=1]
df_fatalities[['FATALITIES','NUMBER OF PERSONS KILLED','NUMBER OF MOTORIST KILLED','NUMBER OF PEDESTRIANS KILLED','NUMBER OF CYCLIST KILLED']]

,FATALITIES,NUMBER OF PERSONS KILLED,NUMBER OF MOTORIST KILLED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST KILLED
39,2.0,1.0,0,1,0
148,2.0,1.0,0,1,0
591,2.0,1.0,0,1,0
605,2.0,1.0,1,0,0
1320,2.0,1.0,0,1,0
...,...,...,...,...,...
2016625,2.0,1.0,1,0,0
2016626,2.0,1.0,1,0,0
2016774,2.0,1.0,1,0,0
2018044,2.0,1.0,1,0,0


In [26]:
df[df['NUMBER OF PERSONS KILLED']>=1]

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,FATALITIES
39,2021-07-09 00:43:00,NaN,NaN,40.720535,-73.888850,"(40.720535, -73.88885)",ELIOT AVENUE,NaN,NaN,0.0,...,NaN,NaN,NaN,4456659,Bus,NaN,NaN,NaN,NaN,2.0
148,2021-12-12 09:09:00,NaN,NaN,40.840360,-73.918070,"(40.84036, -73.91807)",JEROME AVENUE,NaN,NaN,0.0,...,NaN,NaN,NaN,4487210,Taxi,NaN,NaN,NaN,NaN,2.0
591,2021-04-15 15:18:00,BROOKLYN,11209.0,40.620487,-74.029305,"(40.620487, -74.029305)",4 AVENUE,FOREST PLACE,NaN,0.0,...,NaN,NaN,NaN,4408063,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,2.0
605,2021-04-15 22:36:00,NaN,NaN,NaN,NaN,NaN,Trans- Manhattan Expressway,Amsterdam Avenue,NaN,4.0,...,NaN,NaN,NaN,4407693,Sedan,NaN,NaN,NaN,NaN,2.0
1320,2021-04-17 13:31:00,NaN,NaN,40.782463,-73.978830,"(40.782463, -73.97883)",AMSTERDAM AVENUE,NaN,NaN,0.0,...,NaN,NaN,NaN,4408062,E-Bike,NaN,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016625,2023-07-01 00:27:00,QUEENS,11372.0,40.753536,-73.886900,"(40.753536, -73.8869)",80 STREET,34 AVENUE,NaN,3.0,...,NaN,NaN,NaN,4643896,Motorcycle,NaN,NaN,NaN,NaN,2.0
2016626,2023-07-09 09:25:00,QUEENS,11103.0,40.764730,-73.912110,"(40.76473, -73.91211)",28 AVENUE,42 STREET,NaN,0.0,...,NaN,NaN,NaN,4643897,Station Wagon/Sport Utility Vehicle,Moped,NaN,NaN,NaN,2.0
2016774,2023-07-17 22:15:00,BROOKLYN,11236.0,40.632435,-73.888180,"(40.632435, -73.88818)",ROCKAWAY PARKWAY,SKIDMORE AVENUE,NaN,0.0,...,NaN,NaN,NaN,4646703,Sedan,Motorcycle,NaN,NaN,NaN,2.0
2018044,2023-07-22 11:17:00,QUEENS,11429.0,40.705220,-73.727880,"(40.70522, -73.72788)",112 AVENUE,CROSS ISLAND PARKWAY,NaN,0.0,...,NaN,NaN,NaN,4648067,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,2.0


## Likelihood of fatal accidents
#### On average, out of every 1000 accidents, how many have resulted in at least one person dead?

In [31]:
fatal = len(df[df['NUMBER OF PERSONS KILLED']>=1])
fatal

2804

In [43]:
fatal/len(df)*1000

1.3893258747079764

## Missing data
#### What proportion of accidents in the data do not have a Borough code?

In [37]:
missing_borough_count = df['BOROUGH'].isna().sum()

In [38]:
missing_borough_count/len(df)

0.3110890897784957

## Which combinations of vehicles have the most number of accidents

In [41]:
# Group by the vehicle type columns and count the occurrences
com_vehicles = df.groupby(['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']).size()

In [42]:
com_vehicles.sort_values(ascending = False)

VEHICLE TYPE CODE 1                  VEHICLE TYPE CODE 2                
Sedan                                Sedan                                  197944
PASSENGER VEHICLE                    PASSENGER VEHICLE                      193260
Station Wagon/Sport Utility Vehicle  Station Wagon/Sport Utility Vehicle    133780
Sedan                                Station Wagon/Sport Utility Vehicle    123915
Station Wagon/Sport Utility Vehicle  Sedan                                  123812
                                                                             ...  
Pick-up Truck                        UNKNOWN                                     1
                                     UNK L                                       1
                                     U HAU                                       1
                                     Truck                                       1
�MBU                                 Taxi                                        1
Length: 6896, 

***  
## Finished

### Please find more information in the follwing link. Thank you!
*https://github.com/Riccia-Liang/Business_Analytics* 
